In [2]:
import python.data as data
project_folder = "monresovelo"

import pickle 
import pandas as pd

In [3]:
if(not data.check_file("files/"+project_folder+"/data_processed/observations_matched.df")):
    %run data/monresovelo/load_data.py -h #can't use the variable project_folder here, need to be changed manually

In [4]:
data.simplify_gps("files/"+project_folder+"/data_processed/observations_matched.df", 
              "files/"+project_folder+"/data_processed/observations_matched_simplified.df")

In [6]:
data.check_file("files/"+project_folder+"/data_processed/unreachable_routes.tab", [[], [], []])
if(not data.check_file("files/"+project_folder+"/city_graphs/city.ox")):
    %run data/monresovelo/load_city.py -h

Loading city...


/home/lucas/.local/lib/python3.6/site-packages/osmnx/utils_geo.py:335: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for polygon in geometry:
/home/lucas/.local/lib/python3.6/site-packages/osmnx/utils_geo.py:426: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:
/home/lucas/.local/lib/python3.6/site-packages/osmnx/utils_geo.py:426: ShapelyDeprecationWarning: Iteration over multi-part geometries is deprecated and will be removed in Shapely 2.0. Use the `geoms` property to access the constituent parts of a multi-part geometry.
  for poly in multipoly:


In [2]:
data.pathfinding_osmnx("files/"+project_folder+"/data_processed/observations_matched_simplified.df", 
                       "files/"+project_folder+"/data_processed/osmnx_pathfinding.df", 
                       "files/"+project_folder+"/city_graphs/city.ox",
                       "files/"+project_folder+"/data_processed/unreachable_routes.tab")

Route 4880/4881 

In [3]:
data.simplify_gps("files/"+project_folder+"/data_processed/osmnx_pathfinding.df", 
                  "files/"+project_folder+"/data_processed/osmnx_pathfinding_simplified.df")

Simplifying route 4880/4881 

## Distance

In [3]:
data.compute_distance("files/"+project_folder+"/data_processed/observations_matched_simplified.df",
                      "files/"+project_folder+"/distances/distances_observations.tab")
data.compute_distance("files/"+project_folder+"/data_processed/osmnx_pathfinding_simplified.df", 
                      "files/"+project_folder+"/distances/distances_osmnx.tab")

Calculating the distance of route 4880/4881Warning: creating files/monresovelo/distances/distances_observations.tab
Calculating the distance of route 4880/4881Warning: creating files/monresovelo/distances/distances_osmnx.tab


# Mapbox

In [26]:
data.pathfinding_mapbox("files/"+project_folder+"/data_processed/observations_matched_simplified.df", 
                    "files/"+project_folder+"/data_processed/mapbox_pathfinding.df")

In [27]:
data.simplify_gps("files/"+project_folder+"/data_processed/mapbox_pathfinding.df", 
                  "files/"+project_folder+"/data_processed/mapbox_pathfinding_simplified.df")

Simplifying route 4880/4881 